In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
def plot_diversity(data_x, data_y, dirname, run_tag, algorithm, problem):
    plt.figure()
    plt.plot(data_x, data_y, marker='o', linestyle='-', color='b')

    # Add title and labels
    plt.title('Porcentaje de diversidad (población nueva) para entrenamiento '+algorithm+' en '+problem)
    plt.xlabel('Iteraciones')
    plt.ylabel('Porcentaje')
    plt.xticks(data_x)


    # show grid
    plt.grid(True)

    # save plot
    plt.savefig(os.path.join(dirname, "diversity_" + run_tag + "_" + algorithm + "_"+ problem +".png"))
    plt.close()

In [3]:
def dir_plot_diversity(input_dir):
    for dirname, _, filenames in os.walk(input_dir):
        for filename in filenames:
            try:
                # Linux filesystem
                algorithm, problem = dirname.split("/")[-2:]
            except ValueError:
                # Windows filesystem
                algorithm, problem = dirname.split("\\")[-2:]

            if "DIV" in filename:
                run_tag = [s for s in filename.split(".") if s.isdigit()].pop()

                content = []
                with open(os.path.join(dirname, filename), "r") as content_file:
                    for line in content_file:
                        line = line.replace("[", "")
                        line = line.replace("]", "")
                        line = line.replace(",", "")
                        vector = [float(x) for x in line.split()]
                        content.append(vector)
                    div_index = list()
                    div_result = list()
                    for i in range(len(content)):
                        div_index.append(i+1)
                        div_result.append(content[i][1]/content[i][0])
                    plot_diversity(div_index, div_result, dirname, run_tag, algorithm, problem)
                

In [4]:
def plot_2d_pareto(data_x, data_y, dirname, run_tag, algorithm, problem):
    plt.figure()

    referenceFront = np.loadtxt("resources/reference_front/"+problem+".pf", delimiter=' ')
    referenceFront = referenceFront[referenceFront[:, 0].argsort()]


    plt.plot(referenceFront[:,0], referenceFront[:,1], color="red")

    plt.scatter(data_x, data_y, color="black", marker = '.')

    plt.xlabel("f1")
    plt.ylabel("f2")
    plt.title("Frente de Pareto para "+algorithm+" en "+problem)
    plt.legend(["Frente de referencia",algorithm])

    # show grid
    plt.grid(True)

    # save plot
    plt.savefig(os.path.join(dirname, "pareto_" + run_tag + "_" + algorithm + "_"+ problem +".png"))
    plt.close()

In [5]:
def dir_plot_2d(input_dir):
    for dirname, _, filenames in os.walk(input_dir):
        for filename in filenames:
            try:
                # Linux filesystem
                algorithm, problem = dirname.split("/")[-2:]
            except ValueError:
                # Windows filesystem
                algorithm, problem = dirname.split("\\")[-2:]

            if "FUN" in filename:
                run_tag = [s for s in filename.split(".") if s.isdigit()].pop()

                content = []
                with open(os.path.join(dirname, filename), "r") as content_file:
                    for line in content_file:
                        vector = [float(x) for x in line.split()]
                        content.append(vector)
                    f1_objective = [objective[0] for objective in content]
                    f2_objective = [objective[1] for objective in content]
                    plot_2d_pareto(f1_objective, f2_objective, dirname, run_tag, algorithm, problem)
                
    

In [6]:
import re
def read_3d_pf(file):
    reader = open(file, "r")
    lines = reader.readlines()
    reader.close()
    f1_objective = []
    f2_objective = []
    f3_objective = []
    
    for line in lines:
        line = line.strip().split(" ")
        for i in range(len(line)):
            number = re.sub(r'[^0-9.]', '', line[i])
            if number != "":
                if i == 0:
                    f1_objective.append(float(number))
                elif i == 1:
                    f2_objective.append(float(number))
                else:
                    f3_objective.append(float(number))
    return f1_objective, f2_objective, f3_objective

In [7]:
def plot_3d_pareto(data_x, data_y, data_z, dirname, run_tag, algorithm, problem):
    
    fig = plt.figure()

    referenceFront_x , reference_front_y, reference_front_z = read_3d_pf("resources/reference_front/"+problem+".pf")

    ax = fig.add_subplot(111, projection='3d')

    ax.scatter(referenceFront_x, reference_front_y, reference_front_z, color="red", alpha=0.6, s=20, edgecolors='w')
    ax.scatter(data_x, data_y, data_z, color="black")

    ax.set_xlabel('f1')
    ax.set_ylabel('f2')
    ax.set_zlabel('f3')
    ax.set_title("Frente de Pareto para "+algorithm+" en "+problem)
    ax.legend(["Frente de referencia",algorithm])

    # show grid
    plt.grid(True)

    # save plot
    plt.savefig(os.path.join(dirname, "pareto_" + run_tag + "_" + algorithm + "_"+ problem +".png"))
    plt.close()

In [8]:
def dir_plot_3d(input_dir):
    for dirname, _, filenames in os.walk(input_dir):
        for filename in filenames:
            try:
                # Linux filesystem
                algorithm, problem = dirname.split("/")[-2:]
            except ValueError:
                # Windows filesystem
                algorithm, problem = dirname.split("\\")[-2:]

            if "FUN" in filename:
                run_tag = [s for s in filename.split(".") if s.isdigit()].pop()

                content = []
                with open(os.path.join(dirname, filename), "r") as content_file:
                    for line in content_file:
                        vector = [float(x) for x in line.split()]
                        content.append(vector)
                    f1_objective = [objective[0] for objective in content]
                    f2_objective = [objective[1] for objective in content]
                    f3_objective = [objective[2] for objective in content]
                    plot_3d_pareto(f1_objective, f2_objective, f3_objective, dirname, run_tag, algorithm, problem)

In [9]:
#Plot diversity
dir_plot_diversity("data_zdt")
dir_plot_diversity("data_dtlz")

In [10]:
#Plot Pareto 2d ZDT
dir_plot_2d("data_zdt")

In [11]:
#Plot Pareto 3d DTLZ
dir_plot_3d("data_dtlz")